1.	Write a script that solves the dispersion relationship for wavenumber (k = 2pi/L) given the wave period (T = 2pi/omega) and water depth (h).   This will require an iterative solution.

In [ ]:
#the wave solver creates a wave object that determines the critical properties of the wave given period, depth and amplitude
#see wave_model.py for the class
#see testing wave model pdf for tests of this class

class wave_solver:
    grav = 9.80665  #m^2/s
    waveperiod = 0
    wavefrequency = 0

    #changing values as wave moved initialized for deep water
    initial_amplitude = 0
    initial_depth = 0
    initial_wavelength = 0
    initial_wavenumber = 0
    
    
    def __init__(self,measuredperiod,init_depth, measuredamplitude):
        """
        This initializes the wave model object.  It first defines simple immutable properties of the wave
        Next it defines the initial deep water properties using the dispersion relationship
        """
        #set simple properties
        self.initial_amplitude = measuredamplitude
        self.initial_depth = init_depth
        self.waveperiod = measuredperiod
        self.wavefrequency = (2 * np.pi ) / measuredperiod
        
        #calculation other initial properties based on dispersion relationship
        deep_dr = self.dispersion(self.initial_depth)
        self.initial_wavenumber = deep_dr * self.wavefrequency**2 / self.grav
        self.initial_wavelength = (2*np.pi) / self.initial_wavenumber
    
    
    
    def dispersion(self,eff_depth):
        """
        function to calculate dispersion relationship.  
        This calculates at an effective depth so it can be used to generate the initial deep water wave
        and to calculate  the relationship at transitional and shallow depths
        """
        depthratio = self.wavefrequency**2 * eff_depth / self.grav
        assert depthratio > 0
            
        # Guess at dispersion relationship (from https://www.sciencedirect.com/science/article/pii/037838399090032R):
        dr = np.tanh(depthratio ** 0.75) ** (-2.0 / 3.0)
        
        #converge iteratively (strategy based on https://github.com/ChrisBarker-NOAA/wave_utils)
        iter = 0
        f = dr * np.tanh(dr * depthratio) - 1
        while abs(f) > 1e-10:
            qp = dr * depthratio
            fp = qp / (np.cosh(qp) ** 2) + np.tanh(qp)
            dr = dr - f / fp
            f = dr * np.tanh(dr * depthratio) - 1
            iter += 1
            if iter > 200:
                raise RuntimeError("could not converge")
        #set dispersion relationship
        return dr                    

2.	Write a script that calculates the wave-induced horizontal velocity, vertical velocity and pressure, given the wave amplitude, wave period, water depth and height above the bed.  Note, you will need the wavenumber calculation from (1) for this calculation.

In [ ]:
#these three functions are part of the wave_model class so rely on the properties of the wave model object defined above.
#each function requires a sample height and then uses the other properties of the wave to solve for the velocity or pressure
#see wave_model.py for the class
##see testing wave model pdf for test results

    def horizontal_velocity(self,sampleheight):
        #horizontal velocity at a given depth -- data is averaged so leaving out final cos term
        u = self.initial_amplitude * self.wavefrequency * (np.cosh(self.initial_wavenumber * (self.initial_depth + sampleheight)) / np.sinh(self.initial_wavenumber * self.initial_depth))
        return u
    
    
    def vertical_velocity(self,sampleheight):
        #vertical velocity at a given depth -- data is averaged so leaving out final cos term
        w = self.initial_amplitude * self.wavefrequency * (np.sinh(self.initial_wavenumber * (self.initial_depth + sampleheight)) / np.sinh(self.initial_wavenumber * self.initial_depth))
        return w
    
    def wave_pressure(self,sampleheight):
        #pressure at a given depth -- data is averaged so leaving out final cos term
        pr = (self.grav * self.initial_depth) + (self.initial_amplitude * self.grav * (np.cosh(self.initial_wavenumber * (self.initial_depth + sampleheight)) / np.cosh(self.initial_wavenumber * self.initial_depth)))
        return pr


3.	Why does the wave-induced pressure decrease with distance from the surface?  Explain this in terms of the vertical momentum balance.  What are the implications for measuring wave properties using bottom-mounted sensors.

In class we discussed that the variation in true depth due to alternating wave crests and troughs is offset or balanced by the aternating vertical components of the circular motion of the wave.  The net result is that the pressure in deep water effectively becomes the hydrostatic pressure.  This means that a sensor measuring pressure on the bottom would not detect wave induced pressure changes in deep water.

4.	Download the wave data from the Nantucket NDBC buoy (station 44008) for the year 2016.  Apply the script from (1) to this data and plot the wavelength and wave speed.  Apply the script from (2) to this data and plot the wave-induced horizontal and vertical velocities and wave-induced pressure for a location 1 meter above the seafloor.  
a)	How well do the data agree with the deepwater limit?
b)	Are these waves consistent with linear assumptions used in linear wave-theory?


# Please see the buoy data processing pdf and associated data files.

a)  The vertical and horizontal velocities diminish to zero as the measurement location approaches the bottom which supports the deep water limit.  The reduction in velocities matches my expectation from our discussion in class that the elliptical motion would collapse/flatten to a back and forth motion as the measurement location neared the bottom.

b) I believe these waves are consistent with linear assumptions used in linear wave-theory.  